<a href="https://colab.research.google.com/github/LeeDongGeon1996/TensorFlow-study-note/blob/master/my-practice/Experiment_05_Difficulties_I_had_when_learning_MNIST_dataset_using_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiment_05_Difficulties I had when learning MNIST dataset using Keras

아래와 같은 모델이 있을 때, 
 * Keras의 compile, fit을 활용한 학습과
 * Tensorflow의 GradientTape을 이용한 학습

으로 구현을 해보았다.


In [0]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(28, 28, 1)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(10))
    return model

 * 먼저 keras를 활용한 경우 loss function을 직접 만들어 줄 때 주의 할 점이 있었다.
  * loss2_fn(y_true, y_pred)순으로 parameter가 입력이 되어야하며,
  * loss function을 사용할때 
  > * from_logits=True 이면 model을 통과한 prediction값이 확률 분포가 아닌 형태로 주어진다는 것을 알린다. 따라서 내부적으로 확률 분포 형태로 바꾸어 loss값을 계산한다.
    *하지만 from_logits=False인 경우 확률 분포형태로 prediction값이 입력된 것으로 간주한다.
  * 만약 model의 output layer의 activation function이 softmax라면 model의 ouput이 확률분포 형태로 나오게 되므로 False를 사용해도 된다.
  * 하지만 위의 model과 같이 activation function이 없다면 확률 분포ouput이 아니므로 꼭 True값을 주어야한다,
  * 다른 방법으로는 model을 수정하여 output layer의 activation function을 softmax로 주거나
  * 또 다른 방법으로는 loss계산 하기전 prediction값을 softmax에 통과시킨후 계산한다.  


In [0]:
def loss2_fn(labels, images_soft):
  
    #logits = model(images, training=True)
    #images_soft = tf.nn.softmax(images)

    loss = tf.keras.losses.categorical_crossentropy(y_pred=images_soft, y_true=labels, from_logits=True, label_smoothing=0)
    #loss = tf.keras.losses.categorical_crossentropy(y_pred=images, y_true=labels, from_logits=False)
    return loss

model.compile(optimizer=optimizer, loss=loss2_fn, metrics=['accuracy'])
model.fit(x=train_dataset, epochs=15, validation_data=test_dataset)


 * Tensorflow의 GradientdescentTape을 활용한 방법은 아래와 같다. 
  * 별다른 특이한 점은 없었다.

In [0]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0    
    
    for images, labels in train_dataset:
        train(model, images, labels)
        #grads = grad(model, images, labels)                
        #optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(model, images, labels)        
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    checkpoint.save(file_prefix=checkpoint_prefix)

print('Learning Finished!')